In [ ]:
# Ensure notebook is being run from base repository directory
import os, sys
try:
#     os.chdir("/home/{}/forecast_rodeo_ii".format(os.environ["USER"]))
    os.chdir("/Users/quetzal/Documents/code/covid/covid19-forecast-hub")
except Exception as err:
    print(f"Warning: unable to change directory; {repr(err)}")
from src.utils.notebook_util import isnotebook
if isnotebook():
    # Autoreload packages that are modified
    %load_ext autoreload
    %autoreload 2
else:
    from argparse import ArgumentParser
    
# Computational libraries 
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import numpy.matlib
from collections import deque

# os libraries 
import shutil
from datetime import datetime, timedelta
from functools import partial
import copy
import pdb
import os

from utils import *
from file_util import *
from attributes import *

from zoo_of_experts import *
from zoo_of_hinters import *
from zoo_of_losses import *


import matplotlib.pyplot as plt

In [ ]:
#
# Specify model parameters
#
model_name = "online_expert" 

if not isnotebook():
    # If notebook run as a script, parse command-line arguments
    parser = ArgumentParser()
    parser.add_argument("pos_vars",nargs="*")  # gt_id and horizon 
    
    parser.add_argument('--target_dates', '-t', default="std_future")
    parser.add_argument('--expert_models', '-m', default="doy,cfsv2,multillr,llr",
                       help='Comma separated list of models e.g., <doy,cfsv2,llr,multillr>')
    parser.add_argument('--reg', '-r', default="entropic",
                       help='Regularization type, one of: <quadratic>, <entropic> <orig>, <opt_delay>, <plusplus>')
    parser.add_argument('--alg', '-a', default="adahedgefo",
                       help='Online learning algorithm. One of: <ftl>, <ftrl>, <adahedgefo>, <adahedgesr>, <flip_flop>, <rm>, <rmplus>.')
    parser.add_argument('--quarters', '-q', default="False",
                       help='Whether to reset learning at each quarter. If False, learning is run over all target dates.')    
    parser.add_argument('--hint', '-hi', default="None",
                       help='Optimistic hints, only used with adahedgefo. One of: <None>, <doy>, <prev_y>, <trend_y>, <prev_z>, <mean_z>.')  
    parser.add_argument('--delay', '-d', default=0,
                       help='Delay parameter, number of experts to instantiate. String containting an integer >=0.')      
    parser.add_argument('--exp_name', '-e', default="None",
                       help='Experiment name prefix, use "None" if not running an experiment to use standard submodel name.')
    args, opt = parser.parse_known_args()
    
    # Assign variables                                                                                                                                     
    gt_id = get_id_name(args.pos_vars[0]) # "contest_precip" or "contest_tmp2m"                                                                            
    horizon = get_th_name(args.pos_vars[1]) # "34w" or "56w"    
    target_date_str = args.target_dates # target date object
    model_string = args.expert_models # string of expert prediction, comma separated
    reg = args.reg # algorithm regularization 
    alg = args.alg # algorithm 
    reset_quarters = args.quarters # reset quarters 
    hint_type = args.hint # type of optimistic hint
    exp_name = args.exp_name # name of experiment, to be prepended to submodel name
    delay_param = int(args.delay) # delay parameter 
else:
    # Otherwise, specify arguments interactively
    gt_id = "cumm_death" #"contest_precip", "contest_tmp2m"    
    target_date_str = "std_weekly" #"contest_precip", "contest_tmp2m"
    location = 'US'
    quantile = 0.5
    horizon = "1w"
    alg = 'rmplus' # 'ftl', ftrl', 'adahedgefo', 'adahedgesr', 'flip_flop', 'rm', 'rmplus'
    hint_type = "mean_g"
    delay_param = 0
    exp_name = "None" # Set experiment name, use "None" if not running an experiment
    reg = "None" # 'quadratic', 'entropic'
    
'''
Adaptive hinting parameters
'''
hint_alg = 'rmplus' # 'ftl', ftrl', 'adahedgefo', 'adahedgesr', 'flip_flop', 'rm', 'rmplus'
hint_reg = "None"

In [ ]:
"""
Read in ground truth and model predictions for task
"""
# Get location FIPS codes
fips_codes = get_fips_codes() 

# Read ground truth 
printf(f'Loading {gt_id} ground truth')
gt = get_ground_truth(gt_id, location, load_df=True)

# Get target dates
first_date, last_date = get_data_range(gt_id, location=location)
printf(f"Getting target dates from {first_date} to {last_date}.")
target_dates = get_target_dates(target_date_str, first_date, last_date)
target_date_objs = pd.Series(target_dates)

# Store delta between target date and forecast issuance date
start_delta = timedelta(days=get_start_delta(horizon, gt_id))

# Get model predicitons for the task
printf(f'Loading model predictions for {gt_id}, {horizon}, {location}, q{quantile}')
model_pred_df = get_model_predictions(gt_id, horizon, location=location, quantile=quantile, load_df=True)

# Get model list
persistant_models, all_models = get_persistant_models(model_pred_df)
expert_models = list(all_models)
expert_models.sort()
model_string = (',').join(expert_models)
printf(f"Predictions for models loaded:\n {expert_models}")

In [ ]:
"""
Visualize ground truth 
"""
df_gt.plot()
plt.xticks(rotation=90)
plt.title(f'Ground truth {gt_id}')
plt.show()

"""
Visualize avalible models
"""
plt.plot(list(persistant_models.keys()), [len(x) for x in persistant_models.values()])
plt.xticks(rotation=90)
plt.title('Persistant models avaliable by date')
plt.show()

In [ ]:
display(gt)
display(model_pred_df)
len(expert_models)

In [ ]:
"""
Set up online learning parameterization
"""
# Record submodel name for the online expert model
submodel_name = get_submodel_name(
    expert_models=model_string, 
    reg=reg, 
    alg=alg, 
    quarters=reset_quarters,
    hint = hint_type,
    delay=delay_param,
    training_dates=target_date_str,
    exp_name=exp_name)

printf(f"Submodel name {submodel_name}")

if not isnotebook():
    # TODO: need to set this up
    # Save output to log file
    logger = start_logger(model=model_name,submodel=submodel_name,gt_id=gt_id,
                          horizon=horizon,target_dates=target_date_str)
    # Store parameter values in log                                                                                                                        
    params_names = ['gt_id', 'horizon', 'target_date_str',
                    'model_string', 'alg',
                    'reg', 'reset_quarters', 'hint_type',
                    'delay_param', 'exp_name']
    params_values = [eval(param) for param in params_names]
    log_params(params_names, params_values)  

In [ ]:
'''
Instantiate loss
'''
rodeo_loss = MAELoss()
hint_loss = HintingLossTwoNorm()

''' 
Instantiate prototype online learner
'''

# Instantiate expert
num_experts = delay_param + 1
oe = get_expert(alg=alg, loss=rodeo_loss, T=duration//num_experts, 
                expert_list=expert_models, init_method="uniform", reg=reg, 
                init_df=init_df)  
printf(f"-initializing online_expert for with duration {duration//num_experts}")   

'''
Instantiate hint generator
'''
if alg == "rm" or alg == "rmplus":
    regret_hints = True # Provide pseudo-regret hints
else:
    regret_hints = False # Provide cummulative gradient hints

# TODO: delay horizon 
# In order, list of 1 day hints, 12w hints, 34w hint, future hints, etc.
horizon_hints = [["prev_y"], 
                 [hint_type],
                 [hint_type],
                 [hint_type]]    
    
n_hints = [sum(len(x) for x in horizon_hints)]
hint_models = [item + str(i) for i, sublist in enumerate(horizon_hints) for item in sublist]
hint_partition = [i for i, sublist in enumerate(horizon_hints) for item in sublist]

# Initialize 
printf(f"-initializing joint hinter of type {hint_models}")   
hinter = JointHinter(gt_id, horizon_hints, oe, regret_hints)

'''
Instantiate prototype online hinter
'''
oh = get_expert(alg=hint_alg, loss=hint_loss, T=duration//num_experts, 
                expert_list=hint_models, init_method="uniform", reg=hint_reg,
                partition=hint_partition) 

printf(f"-initializing online_expert for with duration {duration//num_experts}")  

In [ ]:
# Store rmses
rmses = pd.Series(index=target_date_objs, dtype='float64')
regret_gt = pd.DataFrame(columns=expert_models, index=target_date_objs, dtype='float64')
regret_gt.fillna(0, inplace=True) # Fill with zeros
online_params = None
online_hint_params = None
expert_pred_df = None
hint_pred_df = None

'''
Initialize online experts
'''
ei = 0
experts_dates = {}
hinters_dates = {}
experts_queue = [None]*(num_experts)
hinters_queue = [None]*(num_experts)
for e in range(num_experts):
    experts_dates[e] = []    
    hinters_dates[e] = []      
    hinters_queue[e] = copy.deepcopy(oh)        
    experts_queue[e] = copy.deepcopy(oe)    
      
'''
Perform online prediction
'''
for target_date_obj in target_date_objs:
    printf(f"\n\n\t----- Starting {target_date_obj}") 
    
    '''
    Get each expert model prediction for the target date and merge into single dataframe.
    Add predictions to the backlog in expert_df
    '''
    
    tic()
    # Convert target date to string
    date_str = datetime.strftime(target_date_obj, '%Y-%m-%d')  
         
    # Get expert predictions for target date and skip target dates if not all experts have a forecast
    merged_df, missing_experts = get_pred_by_date(model_pred_df, date_str, expert_models)    
    toc()
    
    # TODO: need to deal with missing experts
    # if len(missing_experts) > 0 or merged_df.shape[0] < 1:
    #     printf(f"warning: experts {missing_experts} unavailable for target={target_date_obj}; skipping")
    #     continue
    tic()
    
    # Update full expert_df; keep past predictions around to perform updates of w and hints       
    if expert_pred_df is None:
        expert_pred_df = merged_df.copy()
    else:
        expert_pred_df = pd.merge(expert_pred_df, merged_df, 
                         left_index=True, right_index=True, how='outer', 
                         on=expert_models).sort_index()
        
    '''
    Get an avalible expert to make a prediction for target dates. 
    '''
    cur_oe = experts_queue[ei]
    cur_oh = hinters_queue[ei] 
    
    # Update parameter logging
    params = cur_oe.log_params()
    hint_params = cur_oh.log_params()            
    
    # Update parameter set for dates in target range
    if online_params is None:                                                
         online_params = pd.DataFrame(params, index=[target_date_obj])        
    else:                                                                    
        online_params.loc[target_date_obj] = copy.copy(params)
        
    if online_hint_params is None:                                                
         online_hint_params = pd.DataFrame(hint_params, index=[target_date_obj])        
    else:                                                                    
        online_hint_params.loc[target_date_obj] = copy.copy(hint_params)          
    '''
    Update model weights with previously avaliable ground truth; use current 
    prediction to provide optimistic hint, if necessary.
    '''
    pred = None # Initialize prediction
    printf(f"Making target prediction {target_date_obj} using expert {ei} with experience {cur_oe.t}.")  
    
    # Add current target date to outstanding predictions
    experts_dates[ei].append(target_date_obj)

    last_data_date = target_date_obj - start_delta
    printf(f"--Last data date {last_data_date}")

    pdb.set_trace()
    printf(expert_pred_df)
    # Get expert prediction dates that are avalible for updates
    update_dates = [x for x in experts_dates[ei] if x <= last_data_date]
    if len(update_dates) > 1:
        raise ValueError(f"Warning: more than one date in the update queue {len(update_dates)}")
        
    for date in update_dates:
        expert_pred = expert_pred_df.loc[(expert_pred_df.index.get_level_values('target_date') == date)]
        hinter_pred = hint_pred_df.loc[(hint_pred_df.index.get_level_values('target_date') == date)]
        
        if gt.index.get_level_values("target_date").isin([date]).any():   
            printf(f"Updating expert {ei} w/{cur_oe.t} experience with forecast on {target_date_obj}.")
            printf(f"--Outstanding expert predictions: {len(experts_dates[ei])} for expert {ei}")   
            
            d = experts_dates[ei].pop(0) # Remove prediction to be updated from outstanding predictions

            # Check that that oustanding prediction is correct
            if d != date: raise ValueError(f"Bad date {d} != {date}")
                

            ''' Get loss gradient '''       
            grad = cur_oe.loss_gradient(
                     X=expert_pred.to_numpy(copy=False), 
                     y=gt[date].to_numpy(copy=False),
                     w=online_params.loc[date][expert_models].to_numpy(copy=False))

            ''' Update regret ground truth '''
            r = cur_oe.loss_regret(g=grad, w=online_params.loc[date][expert_models].to_numpy(copy=False))
            regret_gt.loc[date, expert_models] = r
               

            ''' Get hint value '''  
            hinter.update_hint_data(g_fb=grad, y_fb=gt[date].to_numpy(copy=False))  
            
            H = hinter.get_hint_matrix(
                        X_all=expert_pred_df,
                        y_all=gt,
                        w_all=online_params[expert_models],
                        last_data_date=last_data_date,
                        os_preds=experts_dates[ei])

            ''' Get hint weights '''
            hint = cur_oh.update_and_predict(
                         X_cur=H,
                         X_fb=hint_pred_df.loc[date].to_numpy(copy=False).T, 
                         y_fb=regret_gt.loc[experts_dates[ei], :].to_numpy(copy=False).sum(axis=0),
                         w_fb=online_hint_params.loc[date, hint_models].to_numpy(copy=False))
            printf(f"Hint: {hint}")  
            
            ''' Get prediction for target date '''
            pred = cur_oe.update_and_predict(
                         X_cur=merged_df[expert_models].to_numpy(copy=False),
                         hint=hint,
                         X_fb=expert_pred.to_numpy(copy=False), 
                         y_fb=gt[date].to_numpy(copy=False),
                         w_fb=online_params.loc[date][expert_models].to_numpy(copy=False))
            print("pred shape:", pred.shape)

            # Remove updated date  
            expert_pred_df.drop(date, level=0, inplace=True)    
                
    # If no data was avaliable for predict and update, make a prediction without feedback
    if pred is None:
        # Before the first feedback
        H = hinter.get_hint_matrix(
                        X_all=expert_pred_df,
                        y_all=gt,
                        w_all=online_params[expert_models],       
                        last_data_date=last_data_date,
                        os_preds=experts_dates[ei])
        
        hint = cur_oh.update_and_predict(X_cur=H)        
        printf(f"Hint grad sum: {hint}")
        '''
        Get prediction for target date without feedback
        '''
        pred = cur_oe.update_and_predict(
                     X_cur=merged_df[expert_models].to_numpy(copy=False),
                     hint=hint
        )
        print("pred shape:", pred.shape)
    '''
    Update hinting historical parameters
    '''
    index = pd.MultiIndex.from_arrays(
        [[target_date_obj]*len(hint_models), hint_models], 
        names=('target_date', 'hint'))
    
    if hint_pred_df is None:
        hint_pred_df = pd.DataFrame(H.T, columns=expert_models, index=index)        
    else:
        df = pd.DataFrame(H.T, columns=expert_models, index=index)                
        hint_pred_df = pd.merge(hint_pred_df, df, on=expert_models,
                         left_index=True, right_index=True, how='outer')
        
    '''
    Mark the current expert as busy until delay_delta days before gt is avalible
    '''
    # Add expert with key providing the next target date for which the expert can make predictions
    ei = (ei+1) % num_experts
    '''
    Save forecasts and update metric logging
    '''
    params = cur_oe.log_params()
    hint_params = cur_oh.log_params()    
    printf(f"-alg params:\n{params}\n")
    printf(f"-hint params:\n{hint_params}\n")        

    # Update parameter set for dates in target range
    if online_params is None:                                                
         online_params = pd.DataFrame(params, index=[target_date_obj])        
    else:                                                                    
        online_params.loc[target_date_obj] = params.values()
    if online_hint_params is None:                                                
         online_hint_params = pd.DataFrame(hint_params, index=[target_date_obj])        
    else:                                                                    
        online_hint_params.loc[target_date_obj] = hint_params.values()        
        
    # Store expert weights
    merged_df['pred'] = pred
    # Save prediction to file in standard format
    save_forecasts(merged_df.reset_index()[["target_date", "lat", "lon", "pred"]],
        model=model_name, 
        submodel=submodel_name, 
        gt_id=gt_id, 
        horizon=horizon, 
        target_date_str=date_str)
    
    # Skip dates without forecast outputs
    if not gt.index.get_level_values(0).isin([target_date_obj]).any():
        printf(f"warning: some features unavailable for target={target_date_obj}; skipping evalution.")
        printf("-mean rmse: {}, running score: {}".format(mean_rmse, mean_rmse_to_score(mean_rmse)))        
        printf("-rmse: {}, score: {}".format(rmse, mean_rmse_to_score(rmse)))        
    else: 
        # Evaluate and store error
        rmse = cur_oe.loss(gt[target_date_obj].to_numpy(), pred)
        rmses.loc[target_date_obj] = rmse
        printf("-rmse: {}, score: {}".format(rmse, mean_rmse_to_score(rmse)))
        mean_rmse = rmses.mean()
        printf("-mean rmse: {}, running score: {}".format(mean_rmse, mean_rmse_to_score(mean_rmse)))
        
# Save rmses in standard format
rmses = rmses.reset_index()
rmses.columns = ['target_date','rmse']

# Get output directory and create if it doesn't exist
output_folder = get_task_metrics_dir(model=model_name, 
                                     submodel=submodel_name, 
                                     gt_id=gt_id, 
                                     horizon=horizon)
make_directories(output_folder)
metric_file_path = f'{output_folder}/rmse-{gt_id}_{horizon}-{target_date_str}.h5'
pandas2hdf(rmses, metric_file_path)


# Log online learning parameters to file
logdir = get_task_forecast_dir(model=model_name, 
                               submodel=submodel_name,
                               gt_id=gt_id, 
                               horizon=horizon)
make_directories(logdir)
param_file_path = f'{logdir}/learning_params-{gt_id}_{horizon}-{target_date_str}.h5'
pandas2hdf(online_params, param_file_path)

hint_param_file_path = f'{logdir}/hint_params-{gt_id}_{horizon}-{target_date_str}.h5'
pandas2hdf(online_hint_params, hint_param_file_path)